In [1]:
import pandas as pd
from datetime import datetime

In [2]:
## This function merges two dataframes based on the column specified (Column name must be the same on both dfs)
## User can specify if it should be an inner, outer, left or right join. Default is outer join.

def simple_merge(df1, df2, join_on, join='outer'):
    if join == 'inner':
        merged = df1.merge(df2, how='inner', on=join_on)
    elif join == 'left':
        merged = df1.merge(df2, how='left', on=join_on)
    elif join == 'right':
        merged = df1.merge(df2, how='right', on=join_on)
    elif join == 'outer':
        merged = df1.merge(df2, how='outer', on=join_on)
    return merged

In [3]:
## This function standardises date columns to a specific format
## For now, only standardise to d/m/y

def std_date_format(df1, date_column, date_format='%d/%m/%Y'):
    if date_format == '%d/%m/%Y':
        df1[date_column] = df1[date_column].apply(lambda x: x if pd.isna(x) else pd.to_datetime(x).strftime('%d/%m/%Y'))
    return df1

In [4]:
# Import the Customer Order Flow data
df_cust = pd.read_csv('customer_order_flow_NUS v1.1 Shared.csv')

# Filter for soybean oil data
# Filter out any 'ERROR' and 'PARTIALLY CANCELLED' contracts
df_cust = df_cust[df_cust['commodity_group'] == 'SOYBEAN OIL']
df_cust = df_cust[df_cust['contract_line_status'].isin(['COMPLETED','OPEN','CLOSED'])]

df_cust.shape

(18582, 28)

In [5]:
# Standardize date formats to dd/mm/yyyy

date_cols = ['monthly_delivery_start_date','fixed_priced_date','delivery_period_from','delivery_period_to','last_pricing_date','monthly_delivery_end_date','contract_issue_date']

for i in date_cols:
    df_cust = std_date_format(df_cust,i)
    
df_cust

,Cargill Org Code,Counter Party Code,monthly_delivery_start_date,commodity_group,monthly_pricing_quantity,fixed_priced_date,internal_external_counterparty,sales_or_purchase,process,contract_line_status,...,commodity_category,commodity_type,counterparty_country,cargill_organization_code,valuation_pool,delivery_period_from,delivery_period_to,last_pricing_date,monthly_delivery_end_date,contract_issue_date
1,S18,B2703,01/08/2020,SOYBEAN OIL,0.04,31/08/2020,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/08/2020,31/08/2020,31/08/2020,31/08/2020,01/04/2020
2,S18,B2228,01/04/2022,SOYBEAN OIL,20.00,20/10/2021,EXTERNAL CGL,SALES,CRUDE,OPEN,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/04/2022,30/04/2022,20/10/2021,30/04/2022,19/10/2021
3,S18,B2570,01/04/2022,SOYBEAN OIL,27.00,16/09/2021,EXTERNAL CGL,SALES,CRUDE,OPEN,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/04/2022,30/04/2022,16/09/2021,30/04/2022,16/09/2021
5,S18,B2703,01/12/2019,SOYBEAN OIL,150.00,19/11/2019,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/12/2019,31/12/2019,19/11/2019,31/12/2019,18/11/2019
8,S18,B2334,01/06/2020,SOYBEAN OIL,0.04,03/06/2020,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/06/2020,30/06/2020,03/06/2020,30/06/2020,23/01/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454329,S18,B2703,01/07/2020,SOYBEAN OIL,0.02,14/07/2020,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/07/2020,31/07/2020,14/07/2020,31/07/2020,01/04/2020
454330,S18,B2228,01/07/2020,SOYBEAN OIL,39.96,23/04/2020,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,01/07/2020,31/07/2020,23/04/2020,31/07/2020,22/04/2020
454331,S18,B3095,19/07/2018,SOYBEAN OIL,0.02,31/07/2018,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,19/07/2018,31/07/2018,31/07/2018,31/07/2018,19/07/2018
454333,S18,B3096,23/04/2021,SOYBEAN OIL,27.38,23/04/2021,EXTERNAL CGL,SALES,CRUDE,COMPLETED,...,OIL SEEDS,SOYA,UNITED KINGDOM,1014,9AZA 093 C2,23/04/2021,30/04/2021,23/04/2021,30/04/2021,23/04/2021


In [6]:
# Filter for Year 2021 based on fixed_priced_date
df_cust_21 = df_cust[df_cust['fixed_priced_date'].str.contains("2021")]

df_cust_21.shape

(2637, 28)

In [7]:
# Group the data by the fixed_priced_date and sum up the pricing_quantity

df_cust_21_g = df_cust_21.groupby('fixed_priced_date')[['pricing_quantity']].sum()
df_cust_21_g['Date'] = df_cust_21_g.index

In [8]:
# Import the data for Financials, Sentiment, Weather, Carbon

df_fswc = pd.read_csv('Combined_Dataset_250322.csv')

df_fswc

,Date,sp500,nadq,crude_oil,dow_jones,usd,sp500_3,nadq_3,crude_oil_3,dow_jones_3,...,Carbon_Volume_K,temperature_max_90,temperature_min_90,temperature_avg_90,temperature_departure_90,HDD_90,CDD_90,precipitation_90,new_snow_90,snow_depth_90
0,1/1/2021,0.508156,0.538435,0.052222,0.383213,0.378217,0.000000,0.000000,0.000000,0.000000,...,5.31,52,39,45.5,-10.3,19,0,0.00,0.0,0
1,2/1/2021,0.477425,0.498742,0.305569,0.513230,0.075903,0.000000,0.000000,0.000000,0.000000,...,5.31,56,32,44.0,-11.3,21,0,0.00,0.0,0
2,3/1/2021,0.051927,0.046249,0.278285,0.180086,-0.129096,0.000000,0.000000,0.000000,0.000000,...,5.31,70,46,58.0,3.2,7,0,0.00,0.0,0
3,4/1/2021,-1.996479,-1.903586,-2.500008,-2.373722,-1.273207,0.508156,0.538435,0.052222,0.383213,...,19.31,80,48,64.0,9.7,1,0,0.00,0.0,0
4,5/1/2021,-1.905114,-1.766484,-2.218646,-2.269819,-1.142662,0.477425,0.498742,0.305569,0.513230,...,19.37,71,51,61.0,7.2,4,0,0.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,27/12/2021,1.804995,1.706067,0.904345,1.392172,2.547886,0.320129,0.221689,0.658306,0.257986,...,3.63,82,56,69.0,10.8,0,4,0.00,0.0,0
361,28/12/2021,1.788124,1.604199,0.954284,1.451544,2.359251,0.106306,0.105995,0.493874,0.154018,...,4.61,85,59,72.0,14.3,0,7,0.00,0.0,0
362,29/12/2021,1.811514,1.586565,1.024928,1.507562,2.346759,-0.156388,-0.112913,0.247835,-0.064215,...,10.44,82,63,72.5,15.3,0,8,0.00,0.0,0
363,30/12/2021,1.761562,1.558509,1.077303,1.451461,2.173116,1.804995,1.706067,0.904345,1.392172,...,5.02,79,64,71.5,14.8,0,7,0.01,0.0,0


In [9]:
# Merge the two datasets together
# For dates with no orders, we will just take Qty = 0

df_combined = simple_merge(df_fswc,df_cust_21_g,'Date','left')
df_combined['pricing_quantity'] = df_combined['pricing_quantity'].fillna(0)

df_combined

,Date,sp500,nadq,crude_oil,dow_jones,usd,sp500_3,nadq_3,crude_oil_3,dow_jones_3,...,temperature_max_90,temperature_min_90,temperature_avg_90,temperature_departure_90,HDD_90,CDD_90,precipitation_90,new_snow_90,snow_depth_90,pricing_quantity
0,1/1/2021,0.508156,0.538435,0.052222,0.383213,0.378217,0.000000,0.000000,0.000000,0.000000,...,52,39,45.5,-10.3,19,0,0.00,0.0,0,0.00
1,2/1/2021,0.477425,0.498742,0.305569,0.513230,0.075903,0.000000,0.000000,0.000000,0.000000,...,56,32,44.0,-11.3,21,0,0.00,0.0,0,0.00
2,3/1/2021,0.051927,0.046249,0.278285,0.180086,-0.129096,0.000000,0.000000,0.000000,0.000000,...,70,46,58.0,3.2,7,0,0.00,0.0,0,0.00
3,4/1/2021,-1.996479,-1.903586,-2.500008,-2.373722,-1.273207,0.508156,0.538435,0.052222,0.383213,...,80,48,64.0,9.7,1,0,0.00,0.0,0,0.00
4,5/1/2021,-1.905114,-1.766484,-2.218646,-2.269819,-1.142662,0.477425,0.498742,0.305569,0.513230,...,71,51,61.0,7.2,4,0,0.00,0.0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,27/12/2021,1.804995,1.706067,0.904345,1.392172,2.547886,0.320129,0.221689,0.658306,0.257986,...,82,56,69.0,10.8,0,4,0.00,0.0,0,0.00
361,28/12/2021,1.788124,1.604199,0.954284,1.451544,2.359251,0.106306,0.105995,0.493874,0.154018,...,85,59,72.0,14.3,0,7,0.00,0.0,0,300.00
362,29/12/2021,1.811514,1.586565,1.024928,1.507562,2.346759,-0.156388,-0.112913,0.247835,-0.064215,...,82,63,72.5,15.3,0,8,0.00,0.0,0,0.00
363,30/12/2021,1.761562,1.558509,1.077303,1.451461,2.173116,1.804995,1.706067,0.904345,1.392172,...,79,64,71.5,14.8,0,7,0.01,0.0,0,163.08


In [10]:
# Output to csv

df_combined.to_csv("Final_Combined_Dataset_250322.csv", index=False)